# Import libraries

In [2]:
import sys
from pathlib import Path
sys.path.insert(0,'..')
import glob
#!pip install openpyxl

In [3]:
import os
import sys
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import h5py

from functools import partial
from multiprocessing import Pool

# sigmon env: /eos/project/l/lhcsm/public/1.5.20.sh
from lhcsmapi.Time import Time
from lhcsmapi.metadata.SignalMetadata import SignalMetadata
from lhcsmapi.pyedsl.dbsignal.post_mortem.PmDbRequest import PmDbRequest
from lhcsmapi.analysis.RbCircuitQuery import RbCircuitQuery

from src.acquisitions.current_voltage_diode_leads_nxcals import CurrentVoltageDiodeLeadsNXCALS
from src.acquisitions.current_voltage_diode_leads_pm import CurrentVoltageDiodeLeadsPM
from src.acquisitions.ee_t_res_pm import EETResPM
from src.acquisitions.ee_u_dump_res_pm import EEUDumpResPM
from src.acquisitions.leads import Leads
from src.acquisitions.pc_pm import PCPM
from src.acquisitions.qh_pm import QHPM
from src.acquisitions.voltage_logic_iqps import VoltageLogicIQPS
from src.acquisitions.voltage_logic_nqps import VoltageLogicNQPS
from src.acquisitions.voltage_nxcals import VoltageNXCALS

`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: htt

# Read the (clean) MP3 file

In [4]:
mp3_fpa_df_raw = pd.read_excel("./RB_TC_extract_2021_11_22.xlsx")
len(mp3_fpa_df_raw)

4409

In [5]:
# First row contains units, 9 rows contain only "Before Notebooks" and "After Notebooks" information, which need to be droped
mp3_fpa_df = mp3_fpa_df_raw.dropna(subset = ['Date (FGC)', 'Circuit Name'])
mp3_fpa_df_raw.iloc[~mp3_fpa_df_raw.index.isin(mp3_fpa_df.index)] #show droped rows

,Circuit Name,Circuit Family,Period,Date (FGC),Time (FGC),FPA Reason,Timestamp_PIC,Delta_t(FGC-PIC),Delta_t(EE_even-PIC),Delta_t(EE_odd-PIC),...,Outer cable type,I_Q_SM18,dI_Q_Acc,dI_Q_LHC,Comment,Analysis performed by,lhcsmapi version,lhcsm notebook version,Unnamed: 46,Unnamed: 47
0,NaN,RB,NaN,2008-04-01 00:00:00,13:14:15,NaN,NaN,[ms],[ms],[ms],...,NaN,[A],[A],[A],NaN,NaN,NaN,NaN,NaN,NaN
1,RB.A12,RB,Before Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Before Notebooks,NaN,NaN,NaN
169,RB.A23,RB,Before Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Before Notebooks,NaN,NaN,NaN
264,RB.A34,RB,Before Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Before Notebooks,NaN,NaN,NaN
545,RB.A45,RB,Before Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Before Notebooks,NaN,NaN,NaN
897,RB.A56,RB,Before Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Before Notebooks,NaN,NaN,NaN
1146,RB.A67,RB,Before Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Before Notebooks,NaN,NaN,NaN
1268,RB.A78,RB,Before Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Before Notebooks,NaN,NaN,NaN
1430,RB.A81,RB,Before Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Before Notebooks,NaN,NaN,NaN
1581,NaN,RB,After Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,After Notebooks,NaN,NaN,NaN


# Find FGC timestamp for each FPA event in MP3 excel

In [6]:
def find_real_fgc_timestamp(circuit_name, fgc_datetime):
    
    fgc_timestamp = Time.to_unix_timestamp(fgc_datetime)
    metadata_fgc = SignalMetadata.get_circuit_signal_database_metadata(
            'RB', circuit_name, 'PC', 'PM', timestamp_query=fgc_timestamp)

    one_sec_in_ns = 1e9 
    start_time = fgc_timestamp - one_sec_in_ns
    end_time = fgc_timestamp + one_sec_in_ns
 
    source_timestamp_fgc = PmDbRequest.find_events(metadata_fgc['source'],
                                                   metadata_fgc['system'],
                                                   metadata_fgc['className'],
                                                   t_start=start_time,
                                                   t_end=end_time)

    return [(circuit_name, el[1]) for el in source_timestamp_fgc]

# getting the list of timestamps
def get_fgc_timestamp(d):
    date_time_str = f"{d['Date (FGC)']} {d['Time (FGC)']}".replace("00:00:00 ","")
    
    real_fgc_timestamps = find_real_fgc_timestamp(d['Circuit Name'], date_time_str)
    if len(real_fgc_timestamps) > 1:
        print(f"Found more than one event for timestamp {date_time_str}")
    elif not real_fgc_timestamps:
        print(f"Found no event for timestamp {date_time_str}")
        return None
    _, real_fgc_timestamp = real_fgc_timestamps[0]
    return int(real_fgc_timestamp)

In [7]:
mp3_fpa_df['timestamp_fgc'] = mp3_fpa_df.apply(get_fgc_timestamp, axis=1)

Found no event for timestamp 2008-08-19 12:13:28
Found no event for timestamp 2008-08-19 18:53:12
Found no event for timestamp 2008-08-19 20:15:07
Found no event for timestamp 2008-08-19 20:15:07
Found no event for timestamp 2008-08-20 18:41:36
Found no event for timestamp 2008-08-22 09:59:25
Found no event for timestamp 2008-08-22 09:59:25
Found no event for timestamp 2008-08-22 09:59:25
Found no event for timestamp 2008-07-08 08:30:10
Found no event for timestamp 2008-07-08 14:39:16
Found no event for timestamp 2008-07-09 17:14:45
Found no event for timestamp 2008-07-11 14:42:00
Found no event for timestamp 2008-07-15 21:20:30
Found no event for timestamp 2008-07-15 21:20:30
Found no event for timestamp 2008-07-15 21:20:30
Found no event for timestamp 2008-07-15 21:20:30
Found no event for timestamp 2008-07-15 21:20:30
Found no event for timestamp 2008-08-09 02:19:48
Found no event for timestamp 2008-09-07 17:34:03
Found no event for timestamp 2009-12-04 11:19:48
Found no event for t

Found no event for timestamp 2008-05-06 18:12:16
Found no event for timestamp 2008-05-06 18:12:16
Found no event for timestamp 2008-05-06 18:12:16
Found no event for timestamp 2008-05-07 17:53:50
Found no event for timestamp 2008-05-07 17:53:50
Found no event for timestamp 2008-05-07 17:53:50
Found no event for timestamp 2008-05-07 17:53:50
Found no event for timestamp 2008-05-07 17:53:50
Found no event for timestamp 2008-05-07 17:53:50
Found no event for timestamp 2008-05-07 17:53:50
Found no event for timestamp 2008-05-09 15:13:38
Found no event for timestamp 2008-05-09 15:13:38
Found no event for timestamp 2008-05-09 15:13:38
Found no event for timestamp 2008-05-09 15:13:38
Found no event for timestamp 2008-05-15 19:43:14
Found no event for timestamp 2008-05-15 19:43:14
Found no event for timestamp 2008-05-16 17:22:27
Found no event for timestamp 2008-05-16 17:22:27
Found no event for timestamp 2008-05-16 17:22:27
Found no event for timestamp 2008-05-16 17:22:27
Found no event for t

Found no event for timestamp 2021-05-22 00:43:20.100000
Found no event for timestamp 2021-05-22 00:43:20.100000
Found no event for timestamp 2021-05-22 00:43:20.100000
Found no event for timestamp 2021-05-22 00:43:20.100000
Found no event for timestamp 2021-05-22 00:43:20.100000
Found no event for timestamp 2021-05-22 00:43:20.100000
Found no event for timestamp 2021-05-22 00:58:40.300000
Found no event for timestamp 2021-05-22 00:58:40.300000
Found no event for timestamp 2021-05-22 00:58:40.300000
Found no event for timestamp 2021-05-22 00:58:40.300000
Found no event for timestamp 2021-05-22 00:58:40.300000
Found no event for timestamp 2021-05-22 00:58:40.300000
Found no event for timestamp 2021-06-15 00:10:53.600000
Found no event for timestamp 2021-06-15 00:10:53.600000
Found no event for timestamp 2021-06-15 00:10:53.600000
Found no event for timestamp 2021-06-15 00:10:53.600000
Found no event for timestamp 2021-06-15 00:10:53.600000
Found no event for timestamp 2021-06-19 00:25:03


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


# Find missing fgc timestamps in mp3 file

In [8]:
mp3_fpa_df_primary_missing = mp3_fpa_df[(mp3_fpa_df.timestamp_fgc.isna()) & (mp3_fpa_df['Nr in Q event']==1)]
print(f"missing primary events: {len(mp3_fpa_df_primary_missing)}")
mp3_fpa_df_primary_missing.Period.value_counts()

missing primary events: 88


HWC 2008    75
HWC 2007     6
HWC 2021     4
HWC 2015     2
HWC 2009     1
Name: Period, dtype: int64

In [9]:
def get_fgc_timestamp_missing(d):
    date_time_str = f"{d['Date (FGC)']} {d['Time (FGC)']}".replace("00:00:00","")
    for t in range(6,24):
        date_time_str_new = date_time_str.replace(" 00:",f" {t}:")
        real_fgc_timestamps = find_real_fgc_timestamp(d['Circuit Name'], date_time_str_new)
        if real_fgc_timestamps:
            print(f"Found event for timestamp {date_time_str_new}")
            _, real_fgc_timestamp = real_fgc_timestamps[0]
            return int(real_fgc_timestamp)

In [10]:
mp3_fpa_df_primary_missing['timestamp_fgc'] = mp3_fpa_df_primary_missing.apply(get_fgc_timestamp_missing, axis=1)

Found event for timestamp 2021-05-22  8:43:20.100000
Found event for timestamp 2021-05-22  9:58:40.300000
Found event for timestamp 2021-06-15  14:10:53.600000
Found event for timestamp 2021-06-19  7:25:03.400000



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [11]:
found_fgc_timestamps_df = mp3_fpa_df_primary_missing["timestamp_fgc"].dropna()
mp3_fpa_df.loc[found_fgc_timestamps_df.index, "timestamp_fgc"] = mp3_fpa_df_primary_missing["timestamp_fgc"].dropna().values

mp3_fpa_df_primary_missing = mp3_fpa_df[(mp3_fpa_df.timestamp_fgc.isna()) & (mp3_fpa_df['Nr in Q event']==1)]
print(f"missing primary events: {len(mp3_fpa_df_primary_missing)}")
mp3_fpa_df_primary_missing.Period.value_counts()

missing primary events: 84



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


HWC 2008    75
HWC 2007     6
HWC 2015     2
HWC 2009     1
Name: Period, dtype: int64

In [12]:
# dropping duplicates of timestamps
print(f"{len(mp3_fpa_df['timestamp_fgc'])} mp3 excel file entries")
mp3_fpa_df_unique = mp3_fpa_df.drop_duplicates(subset=['timestamp_fgc', 'Circuit Name']) ### you need timestamp + circuit !!! 

print(f"{len(mp3_fpa_df_unique)} unique fgc events") # 841 Primary quenches?
print(f"{len(mp3_fpa_df[mp3_fpa_df['Nr in Q event']==1])} mp3 primary quench entries") # 834 primary quenches with correct notes

4399 mp3 excel file entries
845 unique fgc events
918 mp3 primary quench entries


# Select events to download

In [13]:
# Time.to_string(1514761200000000000), Time.to_unix_timestamp('2018-01-01 00:00:00+01:00')
mp3_fpa_df_period = mp3_fpa_df_unique[mp3_fpa_df_unique['timestamp_fgc'] >= 1388530800000000000].reset_index(drop=True) 
len(mp3_fpa_df_period)

828

In [14]:
# use columns where download is not complete
context_path = Path('/eos/project/m/ml-for-alarm-system/private/RB_signals/context_data.csv')
if os.path.exists(context_path):
    df_context = pd.read_csv(context_path)
    dowloaded_fgc_ts = df_context[df_context.download_complete == True].timestamp_fgc.values

    mp3_fpa_df_to_download = mp3_fpa_df_period[~mp3_fpa_df_period.timestamp_fgc.isin(dowloaded_fgc_ts)]
else:
    mp3_fpa_df_to_download = mp3_fpa_df_period
len(mp3_fpa_df_to_download)

828

In [15]:
signal_groups = [CurrentVoltageDiodeLeadsPM, CurrentVoltageDiodeLeadsNXCALS, EETResPM, EEUDumpResPM, Leads, PCPM, QHPM, VoltageLogicIQPS, VoltageLogicNQPS, VoltageNXCALS]

file_dir = Path('/eos/project/m/ml-for-alarm-system/private/RB_signals')

In [ ]:
for index, row in mp3_fpa_df_to_download.iterrows():
    circuit_type = row['Circuit Family']
    circuit_name = row['Circuit Name']
    timestamp_fgc = int(row['timestamp_fgc'])
    
    for signal_group in signal_groups:
        group = signal_group(circuit_type, circuit_name, timestamp_fgc, spark=spark)
        group.to_hdf5(file_dir=file_dir)
        
    group.log_acquisition(log_data={"download_complete": True}, log_path=context_path)

	Querying PM event timestamps for system: QPS, className: DQAMCNMB_PMSTD, source: * from 2014-12-11 21:44:24.580 to 2014-12-11 21:53:34.580


Querying U, I Diode PM:   0%|          | 0/1 [00:00<?, ?it/s]

	Querying PM event timestamps for system: QPS, className: DQAMGNSRB_PMREL, source: B17R1 from 2014-12-11 21:35:14.533 to 2014-12-11 21:55:14.533
	Querying PM event timestamps for system: QPS, className: DQAMCNMB_PMSTD, source: * from 2014-12-11 21:44:24.580 to 2014-12-11 21:51:04.580


Querying U, I Diode leads NXCALS:   0%|          | 0/1 [00:00<?, ?it/s]

	Querying PM event timestamps for system: QPS, className: DQAMSNRB, source: RR17.RB.A12, UA23.RB.A12 from 2014-12-11 21:45:04.580 to 2014-12-11 22:01:54.580
	Querying PM event signal(s) RR17.RB.A12:T_RES_BODY_2, RR17.RB.A12:T_RES_BODY_1, UA23.RB.A12:T_RES_BODY_2, RR17.RB.A12:T_RES_BODY_3, UA23.RB.A12:T_RES_BODY_1... for system: RR17.RB.A12, UA23.RB.A12..., className: DQAMSNRB..., source: QPS... at 2014-12-11 21:45:14.877


...

Some nQPS PM buffers missing. ['B8R1', 'B9R1', 'B10R1', 'B11R1', 'B12R1', 'B13R1', 'B14R1', 'B15R1', 'B16R1', 'B17R1', 'B18R1', 'B19R1', 'B20R1', 'B21R1', 'B22R1', 'B23R1', 'B24R1', 'B25R1', 'B26R1', 'B27R1', 'B28R1', 'B29R1', 'B30R1', 'B31R1', 'B32R1', 'B33R1', 'B34R1', 'B34L2', 'B33L2', 'B32L2', 'B31L2', 'B30L2', 'B29L2', 'B28L2', 'B27L2', 'B26L2', 'B25L2', 'B24L2', 'B23L2', 'B22L2', 'B21L2', 'B20L2', 'B19L2', 'B18L2', 'B17L2', 'B16L2', 'B15L2', 'B14L2', 'B13L2', 'B12L2', 'B11L2', 'B10L2', 'B9L2', 'B8L2'] not found for RB.A12 within [(10, 's'), (400, 's')] around 1418560853860000000.
At least one of the timestamps (nQPS or QDS) is missing, query skipped returning empty pd.DataFrame.


	Querying NXCALS signal(s) MB.B8R1:U_DIODE_RB, MB.A10R1:U_DIODE_RB, MB.A9R1:U_DIODE_RB, MB.A8R1:U_DIODE_RB, MB.B9R1:U_DIODE_RB... from 2014-12-14 13:40:03.860 to 2014-12-14 13:49:13.860
	Querying NXCALS signal(s) MB.A10R1:U_EARTH_RB, MB.C12R1:U_EARTH_RB, MB.A8R1:U_EARTH_RB, MB.B8R1:U_EARTH_RB, MB.B11R1:U_EARTH_RB... from 2014-12-14 13:40:03.860 to 2014-12-14 13:49:13.860
	Querying PM event timestamps for system: QPS, className: DQAMCNMB_PMSTD, source: * from 2014-12-14 22:48:19.920 to 2014-12-14 22:57:29.920


Querying U, I Diode PM:   0%|          | 0/4 [00:00<?, ?it/s]

	Querying PM event timestamps for system: QPS, className: DQAMGNSRB_PMREL, source: B31L2 from 2014-12-14 22:39:09.878 to 2014-12-14 22:59:09.878
	Querying PM event timestamps for system: QPS, className: DQAMCNMB_PMSTD, source: * from 2014-12-14 22:48:19.920 to 2014-12-14 22:54:59.920


Querying U, I Diode leads NXCALS:   0%|          | 0/4 [00:00<?, ?it/s]